In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

1.9.0


### Generate Torque Data and Labels 

In [25]:
x_min = 0
x_max = 10
y_min = -5
y_max = 5

def random_examples(N):
    data = np.random.rand(N, 8) 
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] *= x_max
    data[:,3] *= 0

    data[:,4] *= x_max
    data[:,5] *= y_min
    data[:,6] *= x_max
    data[:,7] *= 0
    
    return data

def perp_only_close(N):
    data = np.random.rand(N,8)
    data[:,0] *= x_max
    data[:,1] *= y_max
    data[:,2] = data[:,0]
    data[:,3] *= 0
    
    data[:,4] = data[:,0]
    data[:,5] = -data[:,1]
    data[:,6] = data[:,0]
    data[:,7] *= 0
    
    return data
    
def perp_only_open(N):
    pass

def same_dist_close(N): 
    pass

def same_dist_open(N):
    pass
    
def same_mag_close(N): 
    pass

def same_mag_open(N):
    pass

N_rand = 100
N_perp_open = 10
N_perp_close = 10
N_dist_close = 10
N_dist_open = 10
N_mag_close = 10
N_mag_open = 10

N = N_rand + N_perp_open + N_perp_close + N_dist_close + N_dist_open + N_mag_close + N_mag_open

#data = np.stack((random_examples(N_rand), )) TODO: Populate with data


In [42]:
np.random.rand(10,1)

array([[0.10532716],
       [0.2157444 ],
       [0.66695597],
       [0.1091522 ],
       [0.57462835],
       [0.95494045],
       [0.15226962],
       [0.2371144 ],
       [0.80606543],
       [0.39437425]])

In [37]:
a = np.array([1,2,3])
b = np.array([4,5,6])
#np.concatenate((a,b),axis=-1)
np.stack((a,b))

array([[1, 2, 3],
       [4, 5, 6]])

In [38]:
N = 100 #Size of train set
x_min = 0
x_max = 10
y_min = -5
y_max = 5

N_closed = 5

#Force Vectors: [(start_x1, start_y1), (end_x1, end_y1),
              #  (start_x2, start_y2), (end_x2, end_y2)]
    
'''
Force Vectors: [start_x1, start_y1, end_x1, end_y1, start_x2, start_y2, end_x2, end_y2]
'''

data = np.random.rand(N, 8) 
data[:,0] *= x_max
data[:,1] *= y_max
data[:,2] *= x_max
data[:,3] *= 0

data[:,4] *= x_max
data[:,5] *= y_min
data[:,6] *= x_max
data[:,7] *= 0

#TODO: Label Data
labels = np.array([i[1] * i[2] + i[5] * i[6] == 0 for i in data]).astype(int)

### Preprocessing
- TODO: Normalize data for input into NN.

In [3]:
print(1 in labels)

False


### Create Model 
- TODO: Create model using NN architecture from tensorflow

In [4]:
print(data.shape)
print(labels.shape)

(100, 8)
(100,)


In [22]:
model = keras.Sequential([
    # keras.layers.Flatten(input_shape=(1,8)),
    keras.layers.Dense(40, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model.fit(data, labels, epochs=5)

Epoch 1/5
100/100 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 2/5
100/100 [==============================] - 0s 61us/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 3/5
100/100 [==============================] - 0s 56us/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 4/5
100/100 [==============================] - 0s 57us/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 5/5
100/100 [==============================] - 0s 69us/step - loss: 0.0000e+00 - acc: 1.0000


### Small Data Model 
- Low complexity (high bias) model for small data
- Same number of layers as features at least (in this case, 8). 